# Modelling exoplanets and host star

To define a star:

In [ ]:
import numpy as np
from ravest.model import Star

star = Star(name="TOI-270", mass=0.386)
print(star)

To do anything interesting, we need to define some planets:

In [ ]:
from ravest.model import Planet

planet_b = Planet(letter="b", basis="p k e w tp", params={"p": 3.3601538, "k": 1.27, "e": 0.034, "w": 0,   "tp": 2458386.2913699285})
planet_c = Planet(letter="c", basis="p k e w tp", params={"p": 5.6605731, "k": 4.16, "e": 0.027, "w": 0.2, "tp": 2458388.315370998})
planet_d = Planet(letter="d", basis="p k e w tp", params={"p": 11.379573, "k": 2.56, "e": 0.032, "w":-0.1, "tp": 2458386.7714448916})

We can see what the radial velocity would be due to the effect of a planet:

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,5)
times = np.linspace(2458774, 2458874, 1000)

plt.figure()
plt.title(planet_d)
plt.xlabel("Time")
plt.ylabel("Radial velocity [m/s]")
plt.plot(times, planet_d.radial_velocity(times))

Let's add the planets to the star, and use that to model the combined radial velocity that we would see, due to the effect of all of the planets.

In [ ]:
star.add_planet(planet_b)
star.add_planet(planet_c)
star.add_planet(planet_d)

plt.figure()
plt.title(star)
plt.xlabel("Time")
plt.ylabel("Radial velocity [m/s]")
plt.plot(times, star.radial_velocity(times))

If we had some observed data, we could see how well it fits the system and each individual planetary signal.

In [ ]:
ti, rv, err = np.loadtxt("example_data/TOI270.txt")
star.phase_plot(ti, rv, err)